In [1]:
import json
from pathlib import Path
import pandas as pd
from ultralytics import YOLO

from utils.autocomplete import yolo_utils as yolo
from utils.autocomplete import markup_yolo as markup
from utils.autocomplete import references_utils as ref

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import pipeline
from utils.extraction.prioritet_extractor import PrioritetExtractor
from tqdm import tqdm
tqdm.pandas()

import time

/opt/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)


In [2]:
# PATH_TO_DETECT_PDF = Path('/data/docs/vh/incoming_2023_feb/') # МЭР
# PATH_TO_DETECT_PDF = Path('/data/docs/vh/latest_docs/') # МЭР
PATH_TO_DETECT_PDF = '/data/docs/mer/vh_full/'

with open('./data/comparison.json', 'r') as f:
    comparison = json.load(f)
    
# CLASSES = ['predict_content', 'predict_date', 'predict_number', 'predict_sender', 'predict_signer']

CLASSES = ['predict_application', 'predict_content', 'predict_date', 'predict_number', 'predict_recipient', 
           'predict_sender', 'predict_signer', 'predict_text']


model_predict_classes = YOLO('/local_data/models/model_yolo_8_classes.pt')
model_handwrite = YOLO('/local_data/models/model_yolo_handwrite.pt')
partners = pd.read_csv('/local_data/assets/partners.csv')
partners_employees = pd.read_csv('/local_data/assets/partners_employees.csv')

# model_predict_classes = YOLO('/data/docs/vh/yolo/models/model_5_classes.pt') # МЭР
# model_handwrite = YOLO('/data/docs/vh/yolo/models/model_yolo_handwrite.pt') # МЭР
# partners = pd.read_csv('/data/assets/partners.csv', encoding='windows-1251') # МЭР
# partners_employees = pd.read_csv('/data/assets/partners_employees.csv', encoding='windows-1251') # МЭР

partners = ref.update_partners(partners)
partners_employees = ref.update_partners_empoyees(partners_employees)

In [3]:
len(comparison), len(partners)

(824, 90)

In [4]:
path_simple = '/local_data/clear_full/00003627-c6cf-48ae-b331-5a311f52c819/1d1aaccd-126a-4991-a1a0-fc32ddb365bf.pdf'

with open(path_simple, 'rb') as f:
    pdf_file = f.read()
    
summarizer = pipeline("summarization", 
                      tokenizer=AutoTokenizer.from_pretrained('/local_data/summarization/data/csebuetnlp_mT5_m2o_russian_crossSum'), 
                      model=AutoModelForSeq2SeqLM.from_pretrained('/local_data/summarization/models/csebuetnlp_mT5_m2o_russian_crossSum/'), 
                      max_length=64)

extractor = PrioritetExtractor()

# get predicts

In [5]:
all_paths = []
c = 0
for path in Path(PATH_TO_DETECT_PDF).iterdir():
    all_paths.append(markup.get_true_path_to_pdf(path, with_medo=True))
    c += 1
    print(c, end='\r')
all_paths = list(filter(None, all_paths))

Error occurred while loading file /data/docs/mer/vh_full/1bc6d4bb-8840-4bbc-80d6-760d4a2c28e8/resolution.json. [Errno 2] No such file or directory: '/data/docs/mer/vh_full/1bc6d4bb-8840-4bbc-80d6-760d4a2c28e8/resolution.json'


In [6]:
# paths = all_paths
paths = all_paths[:2]
len(paths)

2

# TODO 
для получения предикта только для текста в yolo_utils было добавленно условие:  
46  if (predict_class := CLASSES[bbox[0]]) == 'predict_text':  
данное условие игнорирует все предсказания кроме текста.

Переработать условие в условие по всем классам

In [7]:
yolo.get_yolo_autocomplete(pdf_file, model_predict_classes, model_handwrite, comparison, partners, 
                           partners_employees, CLASSES, summarizer)


0: 640x480 1 content, 1 date, 1 number, 1 recipient, 1 sender, 1 signer, 1 text, 106.5ms
Speed: 6.1ms preprocess, 106.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


{'predict_application': {'predict_application': None,
  'predict_application_proba': 0.0},
 'predict_content': {'predict_content': None, 'predict_content_proba': 0.0},
 'predict_date': {'predict_date': 'не определён', 'predict_date_proba': 0.0},
 'predict_number': {'predict_number': 'не определён',
  'predict_number_proba': 0.0},
 'predict_recipient': {'predict_recipient': None,
  'predict_recipient_proba': 0.0},
 'predict_sender': {'predict_sender': 'не определён',
  'predict_sender_proba': 0.0,
  'predict_sender_id': []},
 'predict_signer': {'predict_signer': 'не определён',
  'predict_signer_proba': 0.0,
  'predict_signer_id': []},
 'predict_text': {'predict_text': ' В целях подготовки к заседанию комитета по кадрам и вознаграждениям при наблюдательном совете ВЭБ.РФ, запланированному к проведению 16 октября 2019 г. в 14:00, направляю проект повестки заседания и материалы по вопросу «0 внесении изменений в основы системы оплаты труда работников ВЭБ.РФ».',
  'predict_text_proba': 0.77

In [8]:
yolo.open_pdf(path_simple)

In [9]:
%%time

pred = {}
c = 0
for path in paths:
    start = time.time()
    try:
        pred.update(yolo.get_yolo_autocomplete(path, model_predict_classes, model_handwrite, comparison, 
                                          partners, partners_employees, CLASSES, summarizer))
        c += 1
        print(f'in process: {c}/{len(paths)} s/It: {(time.time() - start):.2f} sec', end='\r')
    except Exception as ex:
        print(path)
        print(ex)


0: 640x480 1 date, 1 number, 1 recipient, 1 sender, 1 text, 101.8ms
Speed: 0.3ms preprocess, 101.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x480 1 signer, 1 text, 101.8ms
Speed: 0.3ms preprocess, 101.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x480 1 content, 1 date, 1 number, 1 recipient, 3 senders, 1 signer, 1 text, 102.1ms
Speed: 0.3ms preprocess, 102.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


CPU times: user 50.6 s, sys: 0 ns, total: 50.6 s
Wall time: 5.54 s


In [10]:
pred

{'59a30eb1-3912-48eb-a3c8-d4fe0544a283': {'predict_application': None,
  'predict_application_proba': 0.0,
  'predict_content': None,
  'predict_content_proba': 0.0,
  'predict_date': 'не определён',
  'predict_date_proba': 0.0,
  'predict_number': 'не определён',
  'predict_number_proba': 0.0,
  'predict_recipient': None,
  'predict_recipient_proba': 0.0,
  'predict_sender': 'не определён',
  'predict_sender_proba': 0.0,
  'predict_signer': 'не определён',
  'predict_signer_proba': 0.0,
  'predict_text': '  Федеральная служба по интеллектуальной собственности (Роспатент) в соответствии с пунктом 3.4 протокола совещания по вопросу создания научно- промышленного кластера двойного назначения в г. Санкт-Петербурге под председательством заместителя председателя коллегии Военно-промышленной комиссии Российской Федерации О.И. Бочкарева от 01.08.2019 № БО-П22-16прВПК сообщает следующее.  Согласно подпункту 3.6 пункта 1 статьи 251 Налогового кодекса Российской Федерации (далее — Кодекс) к дохо

In [11]:
predicts = pd.DataFrame.from_dict(pred, orient='index').reset_index()
predicts.rename(columns={'index':'doc_id'}, inplace=True)

In [12]:
predicts

,doc_id,predict_application,predict_application_proba,predict_content,predict_content_proba,predict_date,predict_date_proba,predict_number,predict_number_proba,predict_recipient,predict_recipient_proba,predict_sender,predict_sender_proba,predict_signer,predict_signer_proba,predict_text,predict_text_proba,predict_sender_id,predict_signer_id,predict_summary
0,59a30eb1-3912-48eb-a3c8-d4fe0544a283,None,0.0,None,0.0,не определён,0.0,не определён,0.0,None,0.0,не определён,0.0,не определён,0.0,Федеральная служба по интеллектуальной собст...,0.907241,[],[],О создании научно-промышленного класстера двой...
1,5af1b91a-167a-40cd-ae0f-d8fbb412de42,None,0.0,None,0.0,не определён,0.0,не определён,0.0,None,0.0,не определён,0.0,не определён,0.0,В соответствии с разделом У Правил формирован...,0.915541,[],[],О направлении документов по объектам капитальн...


In [ ]:
doc_id = '59a30eb1-3912-48eb-a3c8-d4fe0544a283'
path = Path(PATH_TO_DETECT_PDF + doc_id)
path_simple = markup.get_true_path_to_pdf(path, with_medo=True)

yolo.open_pdf(path_simple)

In [ ]:
predicts.iloc[0]['predict_text']

In [ ]:
# predicts.to_pickle('/local_data/data/df_yolo_8_classes.pickle')
predicts.to_pickle('/local_data/summarization/df_yolo_8_classes.pickle')

In [13]:
%%time

pred = {}
c = 0
for path in paths:
    doc_id = path.parents[0].name
    start = time.time()
    try:
        predicts = yolo.get_yolo_predicts(path, model_predict_classes, model_handwrite, CLASSES)
        predicts = yolo.update_predicts(predicts, CLASSES)
        pred[doc_id] = predicts
        c += 1
        print(f'in process: {c}/{len(paths)} s/It: {(time.time() - start):.2f} sec', end='\r')
    except Exception as ex:
        print(path)
        print(ex)


0: 640x480 1 date, 1 number, 1 recipient, 1 sender, 1 text, 102.2ms
Speed: 0.3ms preprocess, 102.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x480 1 signer, 1 text, 100.7ms
Speed: 0.3ms preprocess, 100.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x480 1 content, 1 date, 1 number, 1 recipient, 3 senders, 1 signer, 1 text, 100.1ms
Speed: 0.3ms preprocess, 100.1ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


CPU times: user 5.62 s, sys: 1.23 ms, total: 5.62 s
Wall time: 2.7 s


In [14]:
predicts = pd.DataFrame.from_dict(pred, orient='index').reset_index()
predicts.rename(columns={'index':'doc_id'}, inplace=True)
predicts

,doc_id,predict_text,predict_application,predict_content,predict_date,predict_number,predict_recipient,predict_sender,predict_signer
0,59a30eb1-3912-48eb-a3c8-d4fe0544a283,[ Федеральная служба по интеллектуальной собс...,None,None,None,None,None,None,None
1,5af1b91a-167a-40cd-ae0f-d8fbb412de42,[ В соответствии с разделом У Правил формирова...,None,None,None,None,None,None,None


In [3]:
# predicts = pd.read_pickle('/local_data/summarization/df_yolo_8_classes.pickle')
# predicts.to_pickle('/local_data/data/df_yolo_predicts.pickle')